## SLIM - Cython practice

In [3]:
import numpy as np
import scipy.sparse as sp
from DataParser import DataParser
from Notebooks_utils.data_splitter import train_test_holdout
import time
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [2]:
parser = DataParser()
URM_all = parser.get_URM_all()
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.85)

In [4]:
recommender = SLIMElasticNetRecommender(URM_train)

SLIMElasticNetRecommender: URM Detected 36 (0.45 %) cold users.
SLIMElasticNetRecommender: URM Detected 2051 (7.90 %) cold items.


In [5]:
recommender.fit(l1_ratio=0.5, alpha=1, topK=100)

SLIMElasticNetRecommender: Processed 19407 ( 74.71% ) in 5.00 min. Items per second: 64.67
SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 6.60 min. Items per second: 65.61


In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [7]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 4994 ( 100.00% ) in 2.21 sec. Users per second: 2256


In [8]:
result_dict[10]['MAP']

0.0001370348121449443